In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from time import sleep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.4f}'.format

plt.rcParams['figure.figsize'] = (12, 6)

# Makes graphs crisp. Only use if you don't have a lot of points/lines on your graph.
%config InlineBackend.figure_formats = ['svg', 'retina'] # use svg, then retina if svg is not supported

## Scrape Top WBTC-WETH Uniswap V3 Pools from revert finance

We now scrape this table of [Top Pools](https://revert.finance/#/top-positions?sort=underlying_value&underlying-value-from=20000&apr-from=20&token1=0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2&token0=0x2260fac5e5542a773aa44fbcfedf7c193bc2c599&age-from=14) and put the info in a data frame.

In [2]:
# set up
chrome_options = Options()
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument("disable-infobars")
s = Service('/Applications/chromedriver')

In [3]:
browser = webdriver.Chrome(service=s, options = chrome_options)
url = 'https://revert.finance/#/top-positions?sort=underlying_value&underlying-value-from=20000&apr-from=20&token1=0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2&token0=0x2260fac5e5542a773aa44fbcfedf7c193bc2c599&age-from=14'
browser.get(url)
delay = 20 #secods
WebDriverWait(browser, delay)
sleep(delay) # need to wait until the page is fully loaded before scraping

# scrape page and parse
html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
soup = BeautifulSoup(html, "html.parser")

# close browser
browser.close() 

In [4]:
# extract table header
header1 = [item.text for item in soup.find_all("span", class_='text-gray-20')]
header2 = [item_n.text for item_n in soup.find_all('a', class_='text-gray-20 visited:text-gray-20 hover:text-white active:gray-40')]
header = [header1[0]] + header2[:3] + [header1[1]] + header2[3:]

In [5]:
# extract pool/fees column
pool_fees = [item.text for item in soup.find_all("span", class_='text-gray-30')]

# extract nft_id/owner column
pool_links = ['https://revert.finance'+item.get('href') for item in soup.find_all('a', class_="underline text-green-40 visited:text-green-40 hover:text-green-20 active:green-60")]

In [6]:
# extract PnL, APR, fee APR, value columns
ha = [item.text for item in soup.find_all('div', class_="text-ellipsis overflow-hidden whitespace-nowrap")]

pnls = []
aprs = []
fee_aprs = []
vals = []
for i in range(len(ha)):
    j = i%4
    if j==0:
        pnls.append(ha[i])
    elif j==1:
        aprs.append(ha[i])
    elif j==2:
        fee_aprs.append(ha[i])
    elif j==3:
        vals.append(ha[i])
    else:
        pass

In [7]:
# extract age column
age_days = [item.text for item in soup.find_all('div', class_="border-b border-gray-70 transition-all ease-linear delay-30 duration-150 text-sm h-[60px] group-hover:bg-gray-90 flex items-center group-last:border-transparent group-hover:border-b-gray-70 justify-end pl-4 w-32 text-gray-30")]

In [8]:
# scrape the price range for each of these top pools 
price_rngs = []
for url in pool_links:
    browser = webdriver.Chrome(service=s, options = chrome_options)
    browser.get(url)
    delay = 26 #secods
    WebDriverWait(browser, delay)
    sleep(delay) # need to wait until the page is fully loaded before scraping

    # scrape page and parse
    html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    soup = BeautifulSoup(html, "html.parser")

    # close browser
    browser.close() 
    
    # extract price range
    price_range = [item.text for item in soup.find_all('div', class_='lp-assets') if item.text.__contains__('WBTC/WETH')][0].split(' WBTC/WETH')[0]
    price_rngs.append(price_range)

In [9]:
df = pd.DataFrame({'pool/fees':pool_fees, 'nft_id/owner':pool_links, 'PnL': pnls, 
                   'APR':aprs, 'fee APR':fee_aprs, 'value':vals, 'age':age_days, 'price_range':price_rngs,})
df

,pool/fees,nft_id/owner,PnL,APR,fee APR,value,age,price_range
0,0.30%,https://revert.finance/#/uniswap-position/main...,"$106,365.05",45.17%,30.12%,"$1,046,215.08",60.9 days,12.1445 - 13.6928
1,0.30%,https://revert.finance/#/uniswap-position/main...,"$36,387.81",25.40%,25.50%,"$1,028,372.08",54.9 days,13.6108 - 15.9086
2,0.05%,https://revert.finance/#/uniswap-position/main...,"$13,307.09",46.24%,54.35%,"$483,201.55",21.6 days,14.6416 - 15.3001
3,0.30%,https://revert.finance/#/uniswap-position/main...,"$20,582.74",28.22%,28.81%,"$302,677.70",87.8 days,12.0718 - 15.9086
4,0.05%,https://revert.finance/#/uniswap-position/main...,"$4,206.27",29.88%,37.86%,"$207,121.89",24.7 days,13.3681 - 15.3308
5,0.30%,https://revert.finance/#/uniswap-position/main...,"$3,060.18",23.33%,24.90%,"$183,889.71",26.0 days,13.3681 - 16.6908
6,0.30%,https://revert.finance/#/uniswap-position/main...,"$2,981.93",27.97%,28.23%,"$171,629.71",22.7 days,13.7752 - 16.1006
7,0.30%,https://revert.finance/#/uniswap-position/main...,"$1,985.51",20.34%,27.29%,"$149,806.21",23.9 days,12.9730 - 15.6248
8,0.30%,https://revert.finance/#/uniswap-position/main...,"$2,150.86",35.23%,38.00%,"$149,191.29",14.9 days,14.1947 - 15.0724
9,0.30%,https://revert.finance/#/uniswap-position/main...,"$5,233.24",20.18%,20.32%,"$130,548.66",72.5 days,11.1660 - 14.4525


## Clean the data and save as csv

In [21]:
# clean data
df['PnL'] = df.PnL.str.strip('$').str.replace(',', '').astype(float)
df['APR'] = df.APR.str.strip('%').astype(float)
df['fee APR'] = df['fee APR'].str.strip('%').astype(float)
df['value'] = df.value.str.strip('$').str.replace(',', '').astype(float)
df['age'] = df.age.str.replace(' days', '').astype(float)
df['price_lwr'], df['price_upr'] = zip(*df.price_range.str.split(' - '))

In [37]:
df = df.rename(columns={'nft_id/owner':'url'})
df.columns = df.columns.str.replace('/| ', '_', regex=True)
df = df.drop(columns='price_range')
df

,pool_fees,url,PnL,APR,fee_APR,value,age,price_lwr,price_upr
0,0.30%,https://revert.finance/#/uniswap-position/main...,"106,365.0500",45.1700,30.1200,"1,046,215.0800",60.9000,12.1445,13.6928
1,0.30%,https://revert.finance/#/uniswap-position/main...,"36,387.8100",25.4000,25.5000,"1,028,372.0800",54.9000,13.6108,15.9086
2,0.05%,https://revert.finance/#/uniswap-position/main...,"13,307.0900",46.2400,54.3500,"483,201.5500",21.6000,14.6416,15.3001
3,0.30%,https://revert.finance/#/uniswap-position/main...,"20,582.7400",28.2200,28.8100,"302,677.7000",87.8000,12.0718,15.9086
4,0.05%,https://revert.finance/#/uniswap-position/main...,"4,206.2700",29.8800,37.8600,"207,121.8900",24.7000,13.3681,15.3308
5,0.30%,https://revert.finance/#/uniswap-position/main...,"3,060.1800",23.3300,24.9000,"183,889.7100",26.0000,13.3681,16.6908
6,0.30%,https://revert.finance/#/uniswap-position/main...,"2,981.9300",27.9700,28.2300,"171,629.7100",22.7000,13.7752,16.1006
7,0.30%,https://revert.finance/#/uniswap-position/main...,"1,985.5100",20.3400,27.2900,"149,806.2100",23.9000,12.9730,15.6248
8,0.30%,https://revert.finance/#/uniswap-position/main...,"2,150.8600",35.2300,38.0000,"149,191.2900",14.9000,14.1947,15.0724
9,0.30%,https://revert.finance/#/uniswap-position/main...,"5,233.2400",20.1800,20.3200,"130,548.6600",72.5000,11.1660,14.4525


In [38]:
df.to_csv('univ3_wbtc_weth_lp_top_positions.csv')